   # Convolutional Neural Network

Here we will put together a CNN using Keras. 

### Imports for building models

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense

### Imports for data pre-processing and training 

In [20]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
#https://github.com/jrosebr1/imutils
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os

## LetNet architecture

To start we will use a simple image classifier architecture called LetNet.

Layers:

Input -> Convolution_1 -> Pooling_1 -> Convolution_2 -> Pooling_2 -> Fully_Connected_Hidden_Layer -> Output_Softmax

## Setting up LeNet CNN

In [7]:
#Create a class for the model
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        #Create instance of model
        model = Sequential()
        inputShape = (height, width, depth)
        #Add layers
        
        #First convolutional layer w/ 20 convolutional filters which are 5x5 pixels
        #Padding set to same so input size into the convolution is the same as output
        #Activation can be set in convolutiuonal layer or added separately
        model.add(Conv2D(filters=20, kernel_size=5, padding='same',
                         input_shape=inputShape, activation='relu'))
        #model.add(Activation('relu'))
        #Calculate the max values of pixels in a window size of 2x2 pixels
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        #2nd covolutional layer
        model.add(Conv2D(filters=50, kernel_size=5, padding='same',
                        activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        #Flatten layer serves as a connecter between convolution and densely connected layers
        #Flattens into a single vector
        model.add(Flatten())
        #Dense layer contains 500 fully connected nodes
        model.add(Dense(500, activation='relu'))
        #Can add activation layer separately
        #model.add(Activation('relu'))
        #Output will be number of classes, softmax will yield probability of each class
        model.add(Dense(classes, activation='softmax'))
        
        return model

## Image Pre-processing

In [21]:
#containers for pre-processed image data and class labels
data = []
labels = []

#images directory contains 3 sub-directories: 'poison_ivy', 'poison_oak', 'poison_sumac'
#randomly get image paths and shuffle them
image_paths = sorted(list(paths.list_images('C:\\Users\\jltsa\\Desktop\\Project_2\\test_images')))
random.seed(42)
random.shuffle(image_paths)

#preprocess images to 28x28 pixels as required for LeNet
for image_path in image_paths:
    image = cv2.imread(image_path)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    
    #Extract class labels
    label = image_path.split(os.path.sep)[-2]
    if label == 'poison_ivy':
        label = 0
    elif label == 'poison_oak':
        label = 1
    else:
        label = 2
    labels.append(label)

In [26]:
#Scaling Images

#Scal pixel intensities from 0 to 1
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

## Training CNN

In [17]:
#Initialize global training variables

EPOCHS = 25
#Learning rate
LR = 1e-3
#Batch Size
BS = 25

In [28]:
#Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)


In [31]:
#One hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [33]:
#To increase the amount of training data, build an image generator using data augmentation
aug_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                            height_shift_range=0.1, shear_range=0.2,
                            zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [37]:
#Initialize model
model = LeNet.build(width=28, height=28, depth=3, classes=3)
opt = Adam(lr=LR, decay=LR / EPOCHS)

#if model has 2 classes use loss='binary_crossentropy'
model.compile(loss='categorical_crossentropy', optimizer=opt,
             metrics=['accuracy'])

In [38]:
#train model
model.fit_generator(aug_gen.flow(X_train, y_train, batch_size=BS),
                   validation_data=(X_test, y_test), steps_per_epoch=len(X_train // BS),
                   epochs=EPOCHS, verbose=1)

Epoch 1/25
27/27 [==============================] - 8s 292ms/step - loss: 1.1382 - acc: 0.3222 - val_loss: 1.1028 - val_acc: 0.5000
Epoch 2/25
27/27 [==============================] - 0s 7ms/step - loss: 1.0593 - acc: 0.4549 - val_loss: 1.2570 - val_acc: 0.4000
Epoch 3/25
27/27 [==============================] - 0s 7ms/step - loss: 0.9984 - acc: 0.4973 - val_loss: 1.2703 - val_acc: 0.3000
Epoch 4/25
27/27 [==============================] - 0s 7ms/step - loss: 0.8754 - acc: 0.5848 - val_loss: 1.2990 - val_acc: 0.4000
Epoch 5/25
27/27 [==============================] - 0s 7ms/step - loss: 0.7562 - acc: 0.6538 - val_loss: 1.5162 - val_acc: 0.3000
Epoch 6/25
27/27 [==============================] - 0s 7ms/step - loss: 0.6731 - acc: 0.7042 - val_loss: 1.7903 - val_acc: 0.4000
Epoch 7/25
27/27 [==============================] - 0s 7ms/step - loss: 0.6626 - acc: 0.7196 - val_loss: 1.9283 - val_acc: 0.4000
Epoch 8/25
27/27 [==============================] - 0s 7ms/step - loss: 0.4961 - acc: 0.

In [39]:
#Save model
model.save('poison_plant_model.h5')

In [40]:
#Load model
#model = load_model('poison_plant_model.h5')